In [2]:
from PIL import Image, ImageOps
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import os
import glob


In [3]:
def show_image(title,image):
    image=cv.resize(image,(0,0),fx=0.3,fy=0.3)
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [4]:
def crop_center_from_image(image, percentage):
    if image is None:
        print("NU EXISTA IMAGINEA")
        return None

    if len(image.shape) == 3:  
        height, width, _ = image.shape
    elif len(image.shape) == 2:  
        height, width = image.shape
    else:
        print("Marime Incorecta")
        return None
    
    # Calculate dimensions of the center crop
    crop_width = int(width * percentage)
    crop_height = int(height * percentage)
    
    # Calculate coordinates for the center crop
    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2
    end_x = start_x + crop_width
    end_y = start_y + crop_height
    
    # Crop the image
    cropped_image = image[start_y:end_y, start_x:end_x]
    return cropped_image

In [5]:
def extrage_plansa(image):
    img_blue = cv.cvtColor(image,cv.COLOR_BGR2HSV)
    copy_of_img = image
    # show_image('blue_mask', image)
    
    lower_blue = np.array([40, 150, 0])
    upper_blue = np.array([200, 255, 255])
    
    mask = cv.inRange(img_blue, lower_blue, upper_blue) 
    blue_image = cv.bitwise_and(image, image, mask=mask)
    # show_image('blue_mask', blue_image)
    
    image = cv.cvtColor(blue_image,cv.COLOR_BGR2GRAY)
    
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 3) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.75, image_g_blur, -0.5, 0)
    # show_image('image_sharpened',image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 10, 255, cv.THRESH_BINARY)
    

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(thresh, kernel)
    # show_image('image_threshold',thresh)

    edges =  cv.Canny(thresh ,200,400)
    # show_image('edges',edges)
    contours, _ = cv.findContours(edges,  cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    max_area = 0
   
    for i in range(len(contours)):
        if(len(contours[i]) >3):
            possible_top_left = None
            possible_bottom_right = None
            for point in contours[i].squeeze():
                if possible_top_left is None or point[0] + point[1] < possible_top_left[0] + possible_top_left[1]:
                    possible_top_left = point

                if possible_bottom_right is None or point[0] + point[1] > possible_bottom_right[0] + possible_bottom_right[1] :
                    possible_bottom_right = point

            diff = np.diff(contours[i].squeeze(), axis = 1)
            possible_top_right = contours[i].squeeze()[np.argmin(diff)]
            possible_bottom_left = contours[i].squeeze()[np.argmax(diff)]
            if cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]])) > max_area:
                max_area = cv.contourArea(np.array([[possible_top_left],[possible_top_right],[possible_bottom_right],[possible_bottom_left]]))
                top_left = possible_top_left
                bottom_right = possible_bottom_right
                top_right = possible_top_right
                bottom_left = possible_bottom_left

    width = 2100
    height = 2100
    puzzle = np.array([top_left, top_right, bottom_left, bottom_right], dtype= np.float32)
    destination = np.array([[0,0], [width, 0], [0, height], [width, height]], dtype = np.float32)

    # image_copy = cv.cvtColor(image.copy(),cv.COLOR_GRAY2BGR)
    cv.circle(copy_of_img,tuple(top_left),20,(0,0,255),-1)
    cv.circle(copy_of_img,tuple(top_right),20,(0,0,255),-1)
    cv.circle(copy_of_img,tuple(bottom_left),20,(0,0,255),-1)
    cv.circle(copy_of_img,tuple(bottom_right),20,(0,0,255),-1)
    # show_image("detected corners",copy_of_img)

    M = cv.getPerspectiveTransform(puzzle, destination)
    result = cv.warpPerspective(copy_of_img, M, (width, height), flags = cv.INTER_LINEAR)
    # result = cv.cvtColor(result, cv.COLOR_GRAY2BGR)
    
    cropped_image = crop_center_from_image(result, 0.736)
    # show_image('img',cropped_image)

    
    return cropped_image


In [6]:
def extrage_plansa_alb_negru(image):
    image = extrage_plansa(image)
    image = cv.cvtColor(image,cv.COLOR_BGR2GRAY)
    # show_image('IMAGINE AICI',image)
    
    image_m_blur = cv.medianBlur(image,3)
    image_g_blur = cv.GaussianBlur(image_m_blur, (0, 0), 3) 
    image_sharpened = cv.addWeighted(image_m_blur, 1.7, image_g_blur, -0.75, 0)
    
    # show_image('image_sharpened',image_sharpened)
    _, thresh = cv.threshold(image_sharpened, 25, 255, cv.THRESH_BINARY)

    kernel = np.ones((3, 3), np.uint8)
    thresh = cv.erode(thresh, kernel)
    # show_image('image_thresholded',thresh)
    image = thresh
    return image
    

In [7]:
def extrage_careu(image, output_folder_name):
    # print(len(image.shape))
    # if image is None or len(image.shape) < 3:
    #     print("Imaginea este goală sau invalidă!")
    #     return
     
    height, width = image.shape
    # if height == 0 or width == 0 or channels != 3:
    #      print("Imaginea are dimensiuni invalide sau nu este color!")
    #      return


    # Creează folderul de ieșire dacă nu există
    if not os.path.exists(output_folder_name):
        os.makedirs(output_folder_name)

    # Împarte imaginea în 14x14 pătrate
    num_rows = 14
    num_cols = 14

    # Dimensiunile fiecărui pătrat
    square_height = height // num_rows
    square_width = width // num_cols

    # Inițializează un contor pentru pătrate
    square_index = 0

    # Parcurge fiecare pătrat
    for i in range(num_rows):
        for j in range(num_cols):
            # Coordonatele pătratului
            x_start = j * square_width
            y_start = i * square_height
            x_end = x_start + square_width
            y_end = y_start + square_height

            # Asigură-te că ultimele pătrate acoperă complet imaginea (în caz de rest la divizare)
            if j == num_cols - 1:  # Ultima coloană
                x_end = width
            if i == num_rows - 1:  # Ultimul rând
                y_end = height

            # Extrage pătratul
            square = image[y_start:y_end, x_start:x_end]
            square = crop_center_from_image(square, 0.90)

            # Salvează pătratul în folderul de ieșire
            if square_index < 10:
                output_path = os.path.join(output_folder_name, f"square_00{square_index}.jpg")
            elif square_index < 100:
                output_path = os.path.join(output_folder_name, f"square_0{square_index}.jpg")
            else:
                output_path = os.path.join(output_folder_name, f"square_{square_index}.jpg")
            cv.imwrite(output_path, square)

            square_index += 1





In [8]:
# # Load the image
# img = cv.imread('imagini_auxiliare/04.jpg')
# img=cv.resize(img,(0,0),fx=0.4,fy=0.4)
# # EXTRAGEM PLANSA 
# 
# result = extrage_plansa_alb_negru(img)
# show_image('img',result)
# # result=cv.resize(result,(0,0),fx=0.4,fy=0.4)
# # find_color_values_using_trackbar(result)
# # EXTRAGEM CAREURILE
# 
# careuri = extrage_careu(result,"empty_table")
# # show_image('CAREURI',careuri)


In [9]:
def verif_numar_careu(careu):
    inaltime, latime = careu.shape
    # border for template
    border_image = cv.copyMakeBorder(
        careu,
        top=inaltime ,
        bottom=inaltime ,
        left=latime ,
        right=latime ,
        borderType=cv.BORDER_CONSTANT,
        value=255
    )

    max_corr = -1
    best_match = -1

    template_folder = "imagini_auxiliare/numbers_v2/" #TEMPLATES -->A SE AVEA GRIJA LA MODIFICARE
    for j in range(0, 91):
        template_path = f"{template_folder}{j}.jpg"

        # Citim șablonul + ne asiguram ca daca nu exista poza de la 1 la 91 nu crapa
        img_template = cv.imread(template_path, cv.IMREAD_GRAYSCALE)
        if img_template is None:
            continue

        # calcul corelatie
        corr = cv.matchTemplate(border_image, img_template, cv.TM_CCOEFF_NORMED)
        max_corr_current = np.max(corr)

        # cea mai bună potrivire
        if max_corr_current > max_corr:
            max_corr = max_corr_current
            best_match = j

    print(f"Numărul cel mai potrivit: {best_match} cu corelația: {max_corr:.4f}")
    return best_match


In [10]:
def calcul_pozitie(nr_poza, poz, selected_number):
    # Calculăm poziția
    linie_litere = poz // 14 + 1
    linie_cifre = chr(poz % 14 + 65)  # Convertim restul în literă

    if nr_poza < 10:
        nume_fisier = f"4_0{nr_poza}.txt"
    else:
        nume_fisier = f"4_{nr_poza}.txt"

    if not os.path.exists("rezultate"):
        os.makedirs("rezultate")

    cale_fisier = os.path.join("rezultate", nume_fisier)

    with open(cale_fisier, "w") as fisier:
        continut = f"{linie_litere}{linie_cifre} {selected_number}\n"
        fisier.write(continut)
        
    print(f"{cale_fisier} cu conținut:\n{continut.strip()}")
    

In [11]:
#MAIN ALGORITHM
#PAS 1 --> retine tablca goala + extragem careurile si le memoram
###########################################
# path = "antrenare/1_*.jpg"   #DE SCHIMBAT 
###########################################
tabla_goala = cv.imread('imagini_auxiliare/01.jpg')
tabla_goala = cv.resize(tabla_goala,(0,0),fx=0.4,fy=0.4)
result_tabla_goala = extrage_plansa_alb_negru(tabla_goala)
extrage_careu(result_tabla_goala,"empty_table")

careuri = []
verif = set()
old_tabla = result_tabla_goala

for careu in glob.glob("empty_table/*.jpg"):
    new_careu = cv.imread(careu, cv.IMREAD_GRAYSCALE)  # Convertim în grayscale pentru consistență
    treshold_binary = 127
    # _, new_careu = cv.threshold(new_careu, treshold_binary, 255, cv.THRESH_BINARY)
    careuri.append(new_careu) 


#PAS 2 trecem prin poze si le retinem careurile
# actual_careuri_index = set()
poza_actuala_index = 0
path = "testare/4_*.jpg"
for img_path in glob.glob(path):
    poza_actuala_index += 1
    new_img = cv.imread(img_path)
    new_img = cv.resize(new_img,(0,0),fx=0.4,fy=0.4)
    result = extrage_plansa_alb_negru(new_img)
    # show_image('img',result)
    extrage_careu(result,"actual_table")
    
    
    max_mean_diff = 0
    careuri_poza_de_verif = []
    #Pas 3 --> verificam patratica unde sunt diferentele cele mai mari
    for careu in glob.glob("actual_table/*.jpg"):
        new_careu = cv.imread(careu, cv.IMREAD_GRAYSCALE)  # Convertim în grayscale pentru consistență
        # _, new_careu = cv.threshold(new_careu, treshold_binary, 255, cv.THRESH_BINARY)
        careuri_poza_de_verif.append(new_careu)
    
    
    max_mean_diff = 0
    poz = 0
    index = 0
    for index,careu in enumerate(careuri_poza_de_verif):  # Corectarea: folosim index și careu
        # if index not in actual_careuri_index:  # Comparăm doar indexul
        # 104,105,90,91 sunt patratelele din mijloc si nu poate fi niciodata o piesa pe ele 
        if(index != 104 and index != 105 and index != 90 and index != 91):
            diferenta_medie_careuri = abs(np.mean(careuri_poza_de_verif[index]) - np.mean(careuri[index]))
            if diferenta_medie_careuri > max_mean_diff:
                max_mean_diff = diferenta_medie_careuri
                poz = index
            # actual_careuri_index.add(poz)  # Adăugăm indexul la setul actual_careuri
    verif.add(poz)
    # print(f"A fost detectat elementul pe pozitia : {poz}")
    
    #Pas final -> verificam cu ce numar face match + pozitia exacta pe tabla de sah
    select_number = verif_numar_careu(careuri_poza_de_verif[poz])
    calcul_pozitie(poza_actuala_index,poz,select_number)
    
    # show_image('CAREURI_GOALE',careuri[poz])
    # show_image('CAREURI',careuri_poza_de_verif[poz])

    old_tabla = result
    
    careuri = careuri_poza_de_verif
    
    
    for file in glob.glob("actual_table/*.jpg"):
        os.remove(file)
    
    
# img_test1 = cv.cvtColor(images[0],cv.COLOR_BGR2GRAY)
    

Numărul cel mai potrivit: 4 cu corelația: 0.8864
rezultate\4_01.txt cu conținut:
9G 4
Numărul cel mai potrivit: 7 cu corelația: 0.8748
rezultate\4_02.txt cu conținut:
10G 7
Numărul cel mai potrivit: 25 cu corelația: 0.7804
rezultate\4_03.txt cu conținut:
11G 25
Numărul cel mai potrivit: 35 cu corelația: 0.8852
rezultate\4_04.txt cu conținut:
12G 35
Numărul cel mai potrivit: 63 cu corelația: 0.8871
rezultate\4_05.txt cu conținut:
13G 63
Numărul cel mai potrivit: 6 cu corelația: 0.8998
rezultate\4_06.txt cu conținut:
9H 6
Numărul cel mai potrivit: 1 cu corelația: 0.9051
rezultate\4_07.txt cu conținut:
7I 1
Numărul cel mai potrivit: 1 cu corelația: 0.9133
rezultate\4_08.txt cu conținut:
8I 1
Numărul cel mai potrivit: 2 cu corelația: 0.8928
rezultate\4_09.txt cu conținut:
9I 2
Numărul cel mai potrivit: 24 cu corelația: 0.8974
rezultate\4_10.txt cu conținut:
9F 24
Numărul cel mai potrivit: 20 cu corelația: 0.8530
rezultate\4_11.txt cu conținut:
9E 20
Numărul cel mai potrivit: 2 cu corelația

In [17]:
###JOC

def citire_lista_player(nr_joc):
    file = open(f'antrenare/{nr_joc}_turns.txt', 'r')
    read = file.read().strip().split('\n')
    events = {}
    for stare in read:
        stare = stare.split(' ')
        events[int(stare[1])] = stare[0]
    return events

# Citeste ultima linie din fisier
def citeste_ultima_linie(fisier):
    try:
        with open(fisier, 'r') as file:
            linii = file.readlines()
            if linii:
                return linii[-1].strip() 
    except FileNotFoundError:
        pass
    return None  


def citire_lista_piese(nr_joc):
    mutari_piese = []
    fisiere = sorted(glob.glob(f"rezultate/{nr_joc}_*.txt"))[:50]  #DE SCHIMBAT PATH
    for path in fisiere:
        file = open(path, 'r')
        read = file.read().strip().split(' ')
        read[1] = int(read[1])
        mutari_piese.append(read)
    return mutari_piese


def tabla_initiala():
    tabla_start = [["." for _ in range(14)] for _ in range(14)]
    caractere_speciale = {
        "x3": [(0, 0), (0, 6), (0, 7), (0, 13), (6, 0), (6, 13), (7, 0), (7, 13), (13, 0), (13, 6), (13, 7), (13, 13)],
        "x2": [(1, 1), (1, 12), (2, 2), (2, 11), (3, 3), (3, 10), (4, 4), (4, 9), (12, 1), (12, 12), (11, 2), (11, 11),
               (10, 3), (10, 10), (9, 4), (9, 9)],
        "/": [(1, 4), (1, 9), (4, 1), (4, 12), (9, 1), (9, 12), (12, 4), (12, 9)],
        "+": [(3, 6), (4, 7), (6, 4), (6, 10), (7, 3), (7, 9), (9, 6), (10, 7)],
        "-": [(2, 5), (2, 8), (5, 2), (5, 11), (8, 2), (8, 11), (11, 5), (11, 8)],
        "*": [(3, 7), (4, 6), (6, 3), (6, 9), (7, 4), (7, 10), (9, 7), (10, 6)]
    }
    for caracter, pozitii in caractere_speciale.items():
        for i, j in pozitii:
            tabla_start[i][j] = caracter
    tabla_start[6][6] = 1
    tabla_start[6][7] = 2
    tabla_start[7][6] = 3
    tabla_start[7][7] = 4
    return tabla_start


def afiseaza_tabla(tabla):
    for linie in tabla:
        print(" ".join(map(str, linie)))


def transform_to_coordinates(pozitie):
    numar = ''.join(filter(str.isdigit, pozitie))
    litera = ''.join(filter(str.isalpha, pozitie))

    j = ord(litera.upper()) - 65
    i = int(numar) - 1

    return i, j


tabla = tabla_initiala()
# afiseaza_tabla(tabla)

piese = citire_lista_piese(1)
events = citire_lista_player(1)


p1_score = 0
p2_score = 0
tura_score = 0
actual_player = 0
last_stop = 0
path_raspuns_scor = "scores.txt"

for index, new_piesa in enumerate(piese):

    if index + 1 in events:
        actual_player = events[index + 1][-1:]
        # print(actual_player)
        if index==0:
            last_stop = 0
        if events[index + 1] == "Player1" and index!=0:
            with open(path_raspuns_scor, 'a') as file:   #NU INTELEG DE CE SE GENERAZA ASA GREU folderul cu scoruri(Recomand verificat folder/ restart pycharm)
                file.write(f"Player2 {last_stop+1} {tura_score}\n")
                print(f"Player2 {last_stop+1} {tura_score}")
                last_stop = index
            tura_score = 0
        if events[index + 1] == "Player2" and index!=0:
            with open(path_raspuns_scor, 'a') as file:
                file.write(f"Player1 {last_stop+1} {tura_score}\n")   #NU INTELEG DE CE SE GENERAZA ASA GREU folderul cu scoruri(Recomand verificat folder/ restart pycharm)
                print(f"Player1 {last_stop+1} {tura_score}")
                last_stop = index
            tura_score = 0
            
    i, j = transform_to_coordinates(new_piesa[0])
    simbol_de_pe_tabla = tabla[i][j]
    tabla[i][j] = int(new_piesa[1])
    new_piesa_tabla = tabla[i][j]
    simbol_calcul = 'x'
    if (i >= 2):
        if (isinstance(tabla[i - 1][j], int) and isinstance(tabla[i - 2][j], int)):
            simbol_calcul = 'x'
            piesa_1 = int(tabla[i - 1][j])
            piesa_2 = int(tabla[i - 2][j])
            piesa_pusa_nou = new_piesa_tabla
            if (piesa_pusa_nou == (piesa_1 + piesa_2)):
                simbol_calcul = '+'
            elif (piesa_pusa_nou == abs(piesa_1 - piesa_2)):
                simbol_calcul = '-'
            elif (piesa_pusa_nou == (piesa_1 * piesa_2)):
                simbol_calcul = '*'
            if(piesa_1 != 0 and piesa_2 != 0 and piesa_pusa_nou !=0):
                if(piesa_pusa_nou == abs(piesa_1 / piesa_2) or (piesa_pusa_nou == abs(piesa_2 / piesa_1))):
                    simbol_calcul = '/'

            if (simbol_de_pe_tabla == "x3" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 3
            elif (simbol_de_pe_tabla == "x2" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 2
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == "."):
                tura_score += tabla[i][j]
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == simbol_calcul):
                tura_score += tabla[i][j]

    

    if (i <= 11):
        if (isinstance(tabla[i + 1][j], int) and isinstance(tabla[i + 2][j], int)):
            simbol_calcul = 'x'
            piesa_1 = int(tabla[i + 1][j])
            piesa_2 = int(tabla[i + 2][j])
            piesa_pusa_nou = new_piesa_tabla
            if (piesa_pusa_nou == (piesa_1 + piesa_2)):
                simbol_calcul = '+'
            elif (piesa_pusa_nou == abs(piesa_1 - piesa_2)):
                simbol_calcul = '-'
            elif (piesa_pusa_nou == (piesa_1 * piesa_2)):
                simbol_calcul = '*'
            if(piesa_1 != 0 and piesa_2 != 0 and piesa_pusa_nou !=0):
                if(piesa_pusa_nou == abs(piesa_1 / piesa_2) or (piesa_pusa_nou == abs(piesa_2 / piesa_1))):
                    simbol_calcul = '/'
                
            # print(simbol_calcul, simbol_de_pe_tabla)
            if (simbol_de_pe_tabla == "x3" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 3
            elif (simbol_de_pe_tabla == "x2" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 2
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == "."):
                tura_score += tabla[i][j]
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == simbol_calcul):
                tura_score += tabla[i][j]


    if (j >= 2):
        if (isinstance(tabla[i][j - 1], int) and isinstance(tabla[i][j - 2], int)):
            simbol_calcul = 'x'
            piesa_1 = int(tabla[i][j - 1])
            piesa_2 = int(tabla[i][j - 2])
            piesa_pusa_nou = new_piesa_tabla
            if (piesa_pusa_nou == (piesa_1 + piesa_2)):
                simbol_calcul = '+'
            elif (piesa_pusa_nou == abs(piesa_1 - piesa_2)):
                simbol_calcul = '-'
            elif (piesa_pusa_nou == (piesa_1 * piesa_2)):
                simbol_calcul = '*'
            if(piesa_1 != 0 and piesa_2 != 0 and piesa_pusa_nou !=0):
                if(piesa_pusa_nou == abs(piesa_1 / piesa_2) or (piesa_pusa_nou == abs(piesa_2 / piesa_1))):
                    simbol_calcul = '/'
                
            if (simbol_de_pe_tabla == "x3" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 3
            elif (simbol_de_pe_tabla == "x2" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 2
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == "."):
                tura_score += tabla[i][j]
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == simbol_calcul):
                tura_score += tabla[i][j]


    if (j <= 11):
        if (isinstance(tabla[i][j + 1], int) and isinstance(tabla[i][j + 2], int)):
            simbol_calcul = 'x'
            piesa_1 = int(tabla[i][j + 1])
            piesa_2 = int(tabla[i][j + 2])
            piesa_pusa_nou = new_piesa_tabla
            # print(piesa_pusa_nou,piesa_1,piesa_2)
            if (piesa_pusa_nou == (piesa_1 + piesa_2)):
                simbol_calcul = '+'
            elif (piesa_pusa_nou == abs(piesa_1 - piesa_2)):
                simbol_calcul = '-'
            elif (piesa_pusa_nou == (piesa_1 * piesa_2)):
                simbol_calcul = '*'
            if(piesa_1 != 0 and piesa_2 != 0 and piesa_pusa_nou !=0):
                if(piesa_pusa_nou == abs(piesa_1 / piesa_2) or (piesa_pusa_nou == abs(piesa_2 / piesa_1))):
                    simbol_calcul = '/'

            if (simbol_de_pe_tabla == "x3" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 3
            elif (simbol_de_pe_tabla == "x2" and simbol_calcul != 'x'):
                tura_score += tabla[i][j] * 2
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == "."):
                tura_score += tabla[i][j]
            elif (simbol_calcul != 'x' and simbol_de_pe_tabla == simbol_calcul):
                tura_score += tabla[i][j]


ultima_linie = citeste_ultima_linie(path_raspuns_scor)
if ultima_linie.startswith("Player1"):
    next_player = "Player2"
else:
    next_player = "Player1"
with open(path_raspuns_scor, 'a') as file:
    file.write(f"{next_player} {last_stop+1} {tura_score}\n")
    print(f"{next_player} {last_stop+1} {tura_score}")
                

    

['9H', '8']
['10H', '32']
['11H', '40']
['12H', '72']
['6H', '8']
['5H', '10']
['4H', '60']
['3H', '8']
['2H', '10']
['1H', '18']
['6G', '3']
['6F', '11']
['6E', '14']
['6I', '5']
['6J', '3']
['6K', '15']
['6L', '12']
['7I', '3']
['8I', '7']
['7J', '6']
['5J', '9']
['7K', '9']
['7L', '3']
['6D', '3']
['9I', '10']
['10I', '17']
['10J', '49']
['8J', '9']
['8K', '63']
['9K', '54']
['5L', '36']
['4L', '48']
['6M', '27']
['4J', '6']
['8F', '7']
['3J', '3']
['2J', '2']
['1J', '5']
['11I', '7']
['12I', '10']
['13I', '70']
['14I', '60']
['9G', '2']
['10G', '5']
['9F', '10']
['10F', '3']
['10E', '8']
['10D', '5']
['9E', '8']
['11E', '0']
Player1 1 152
Player2 5 72
Player1 11 28
Player2 14 35
Player1 18 16
Player2 21 30
Player1 24 128
Player2 28 126
Player1 31 111
Player2 34 13
Player1 36 10
Player2 39 147
Player1 43 38
Player2 48 13


In [22]:
from PIL import Image

# Specifică path-ul imaginii
image_path = "testare/1_01.jpg"

# Încarcă și afișează imaginea
img = Image.open(image_path)
img.show()